In [10]:
import requests
from pathlib import Path
import sys

def search_pdb_for_first_hit(protein_name: str):
    """
    Search RCSB PDB and return the first result found.
    """
    print(f"Searching PDB for the first hit of: {protein_name}")
    print("=" * 70)
    
    query = {
        "query": {
            "type": "terminal",
            "service": "text",
            "parameters": {
                "attribute": "struct.title",
                "operator": "contains_phrase",
                "value": protein_name
            }
        },
        "return_type": "entry",
        "request_options": {
            "return_all_hits": False,
            "results_content_type": ["experimental"],
            "sort": [{"sort_by": "score", "direction": "desc"}]
        }
    }
    
    url = "https://search.rcsb.org/rcsbsearch/v2/query"
    
    try:
        response = requests.post(url, json=query, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        result_set = data.get('result_set', [])
        if not result_set:
            print(f"✗ No structures found for '{protein_name}'")
            return None
            
        first_pdb_id = result_set[0]['identifier']
        print(f"✓ Found first hit: {first_pdb_id}\n")
        return first_pdb_id
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error searching PDB: {e}")
        return None


def remove_ligands_from_pdb(pdb_content: str, keep_chain: str = 'A') -> tuple[str, int, int, int]:
    """
    Remove ligands (HETATM) and keep only a single chain (default: A).
    Also removes CONECT records for removed atoms.
    Returns cleaned content, number of HETATM removed, ATOM kept, and chains removed.
    """
    lines = pdb_content.split('\n')
    cleaned_lines = []
    hetatm_count = 0
    atom_count = 0
    chain_removed = set()
    conect_count = 0

    atom_serials = set()

    # First pass: collect valid atom serials (chain A only)
    for line in lines:
        if line.startswith('ATOM'):
            try:
                chain_id = line[21].strip()
                serial = int(line[6:11].strip())
                if chain_id == keep_chain:
                    atom_serials.add(serial)
            except (ValueError, IndexError):
                pass

    # Second pass: filter
    for line in lines:
        if line.startswith('ATOM'):
            chain_id = line[21].strip()
            if chain_id == keep_chain:
                cleaned_lines.append(line)
                atom_count += 1
            else:
                chain_removed.add(chain_id)

        elif line.startswith('HETATM'):
            hetatm_count += 1
            continue

        elif line.startswith('CONECT'):
            try:
                parts = line.split()
                if len(parts) > 1:
                    serials = [int(x) for x in parts[1:] if x.isdigit()]
                    if all(s in atom_serials for s in serials):
                        cleaned_lines.append(line)
                    else:
                        conect_count += 1
            except (ValueError, IndexError):
                conect_count += 1
                continue

        elif line.startswith(('HEADER', 'TITLE', 'COMPND', 'SOURCE', 'KEYWDS',
                             'EXPDTA', 'AUTHOR', 'REVDAT', 'JRNL', 'REMARK',
                             'SEQRES', 'MODRES', 'HELIX', 'SHEET', 'CRYST1',
                             'ORIGX', 'SCALE', 'MTRIX', 'MODEL', 'ENDMDL',
                             'MASTER', 'END', 'TER')):
            cleaned_lines.append(line)

    print(f"  → Removed {hetatm_count} HETATM records (ligands/waters)")
    print(f"  → Kept {atom_count} ATOM records (chain {keep_chain})")
    if chain_removed:
        print(f"  → Removed chains: {', '.join(sorted(chain_removed))}")
    if conect_count > 0:
        print(f"  → Removed {conect_count} CONECT records for ligands/other chains")

    return '\n'.join(cleaned_lines), hetatm_count, atom_count, len(chain_removed)


def download_pdb_file(pdb_id: str, output_dir: str = "proteins", remove_ligands: bool = True, keep_chain: str = 'A') -> Path:
    """Download PDB file from RCSB PDB and optionally remove ligands and keep only chain A."""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    pdb_id = pdb_id.strip().upper()
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    
    if remove_ligands:
        output_file = output_path / f"{pdb_id}_chain{keep_chain}_clean.pdb"
        print(f"Downloading PDB file for {pdb_id} (keeping chain {keep_chain}, removing ligands)...")
    else:
        output_file = output_path / f"{pdb_id}.pdb"
        print(f"Downloading PDB file for {pdb_id}...")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        content = response.text
        
        if remove_ligands:
            cleaned_content, hetatm_removed, atoms_kept, chains_removed = remove_ligands_from_pdb(content, keep_chain)
            content = cleaned_content
        
        with open(output_file, 'w') as f:
            f.write(content)
        
        file_size = output_file.stat().st_size / 1024
        print(f"✓ Successfully downloaded: {output_file} ({file_size:.2f} KB)\n")
        return output_file
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"✗ Error: PDB ID '{pdb_id}' not found")
        else:
            print(f"✗ HTTP Error: {e}")
        return None
    except Exception as e:
        print(f"✗ Error downloading file: {e}")
        return None


def download_fasta_file(pdb_id: str, output_dir: str = "proteins") -> Path:
    """Download FASTA sequence for a given PDB ID"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    pdb_id = pdb_id.strip().upper()
    url = f"https://www.rcsb.org/fasta/entry/{pdb_id}"
    output_file = output_path / f"{pdb_id}.fasta"
    
    print(f"Downloading FASTA sequence for {pdb_id}...")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        if not response.text.strip().startswith(">"):
            print(f"✗ No valid FASTA data found for {pdb_id}")
            return None
        
        with open(output_file, 'w') as f:
            f.write(response.text)
        
        num_lines = len(response.text.splitlines())
        print(f"✓ Successfully downloaded: {output_file} ({num_lines} lines)\n")
        return output_file
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error downloading FASTA: {e}")
        return None


def download_first_protein_structure(protein_name: str, output_dir: str = "proteins", remove_ligands: bool = True, keep_chain: str = 'A'):
    """
    Search for a protein, find the first PDB ID, and download its PDB and FASTA files.
    Optionally remove ligands and keep only a single chain.
    """
    print("\n" + "=" * 70)
    print(f"STEP 1: Find and Download PDB/FASTA for '{protein_name}'")
    print("=" * 70 + "\n")
    
    pdb_id = search_pdb_for_first_hit(protein_name)
    
    if not pdb_id:
        print("=" * 70)
        print("Process stopped: No PDB ID was found.")
        print("=" * 70)
        return
    
    print("=" * 70)
    print(f"STEP 2: Downloading files for PDB ID: {pdb_id}")
    print("=" * 70 + "\n")
    
    pdb_file = download_pdb_file(pdb_id, output_dir, remove_ligands=remove_ligands, keep_chain=keep_chain)
    fasta_file = download_fasta_file(pdb_id, output_dir)
    
    print("=" * 70)
    print("SUMMARY")
    print("=" * 70)
    if pdb_file or fasta_file:
        print(f"  ✓ Process complete for {pdb_id}.")
        if pdb_file:
            print(f"    PDB file saved to: {pdb_file}")
            print(f"    → Ligands removed, only chain {keep_chain} retained")
        if fasta_file:
            print(f"    FASTA file saved to: {fasta_file}")
    else:
        print(f"  ✗ Failed to download files for {pdb_id}.")
    print("=" * 70 + "\n")


# Example usage
if __name__ == "__main__":
    print("\n" + "🔬 " * 20)
    protein_name = "COX-2"
    download_first_protein_structure(
        protein_name=protein_name,
        output_dir="proteins",
        remove_ligands=True,  # remove ligands/water/ions
        keep_chain='A'        # keep only chain A
    )



🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 🔬 

STEP 1: Find and Download PDB/FASTA for 'COX-2'

Searching PDB for the first hit of: COX-2
✓ Found first hit: 5FDQ

STEP 2: Downloading files for PDB ID: 5FDQ

  → Removed 1417 HETATM records (ligands/waters)
  → Kept 4483 ATOM records (chain A)
  → Removed chains: B
  → Removed 294 CONECT records for ligands/other chains
✓ Successfully downloaded: proteins/5FDQ_chainA_clean.pdb (421.21 KB)

✓ Successfully downloaded: proteins/5FDQ.fasta (2 lines)

SUMMARY
  ✓ Process complete for 5FDQ.
    PDB file saved to: proteins/5FDQ_chainA_clean.pdb
    → Ligands removed, only chain A retained
    FASTA file saved to: proteins/5FDQ.fasta



In [11]:
import subprocess

# Define arguments
input_folder = "proteins/"
output_folder = "my_analysis_folder"
map_type = "0"
resolution = "600"
plot_type = "png"

# Construct command as list
cmd = [
    "ramplot", "pdb",
    "-i", input_folder,
    "-o", output_folder,
    "-m", map_type,
    "-r", resolution,
    "-p", plot_type
]

# Run the command
try:
    result = subprocess.run(cmd, check=True, text=True, capture_output=True)
    print("✅ RAMPlot completed successfully!")
    print("STDOUT:\n", result.stdout)
except subprocess.CalledProcessError as e:
    print("❌ Error running RAMPlot:")
    print("STDERR:\n", e.stderr)


✅ RAMPlot completed successfully!
STDOUT:
 Input Directory: proteins/

Output Directory: my_analysis_folder

Plot Resolutions: 600

Plot File Type: png

Plot Ramachandran Map : 2D & 3D All
Torsion Angle Calculation 
5FDQ.fasta
5FDQ_chainA_clean.pdb
2D Ramachandran Plot All
3D Ramachandran Plot All
2D Ramachandran Plot
3D Ramachandran Plot 
[['proteins//5FDQ_chainA_clean.pdb Chain: A ResNum:34 ResName:HIS'
  '5FDQ_chainA_clean' 'A' ... 'Trans' 'No' 'General']
 ['proteins//5FDQ_chainA_clean.pdb Chain: A ResNum:35 ResName:PRO'
  '5FDQ_chainA_clean' 'A' ... 'Trans' 'No' 'General']
 ['proteins//5FDQ_chainA_clean.pdb Chain: A ResNum:36 ResName:CYS'
  '5FDQ_chainA_clean' 'A' ... 'Trans' 'No' 'General']
 ...
 ['proteins//5FDQ_chainA_clean.pdb Chain: A ResNum:579 ResName:SER'
  '5FDQ_chainA_clean' 'A' ... 'Trans' 'No' 'General']
 ['proteins//5FDQ_chainA_clean.pdb Chain: A ResNum:580 ResName:PHE'
  '5FDQ_chainA_clean' 'A' ... 'Trans' 'No' 'General']
 ['proteins//5FDQ_chainA_clean.pdb Chain: A Re

In [12]:
import requests
import time
import sys
import os
import csv
import re

# --- 1. SET YOUR INPUTS HERE ---

# Get your API token from: https://swissmodel.expasy.org/token
API_TOKEN = "9e8b3ac03b851bb3834cdb311045c78021087d1d" 

# Set the path to your .fasta file
FASTA_FILE_PATH = "proteins/5FDQ.fasta" # Your file path

# Set the path to your CSV file containing the Favoured percentage
CSV_FILE_PATH = "my_analysis_folder/Analysis.csv"  # Change this to your CSV file path

# Give your job a title (you can change this)
PROJECT_TITLE = "My_Protein_Homology_Model"

# --- 2. CHECK FAVOURED PERCENTAGE FROM CSV ---
def extract_favoured_percentage(csv_path):
    """
    Extract the Favoured percentage from CSV file.
    Looks for pattern like: Favoured: ,232,(85.294%)
    """
    try:
        with open(csv_path, 'r') as f:
            content = f.read()
            
            # Search for pattern: Favoured: ,xxx,(yy.yyy%)
            match = re.search(r'Favoured:\s*,\d+,\((\d+\.?\d*)%\)', content)
            
            if match:
                percentage = float(match.group(1))
                return percentage
            else:
                print(f"Error: Could not find 'Favoured:' percentage in {csv_path}")
                print("Expected format: Favoured: ,XXX,(YY.YYY%)")
                return None
                
    except FileNotFoundError:
        print(f"Error: CSV file not found at '{csv_path}'")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

# Check the Favoured percentage
print("Checking Favoured percentage from CSV...")
favoured_percent = extract_favoured_percentage(CSV_FILE_PATH)

if favoured_percent is None:
    print("Failed to extract Favoured percentage. Exiting.")
    sys.exit(1)

print(f"Favoured percentage found: {favoured_percent}%")

# Check if we should proceed
if favoured_percent >= 90.0:
    print(f"\nFavoured percentage ({favoured_percent}%) is >= 90%.")
    print("SWISS-MODEL execution is NOT required. Exiting.")
    sys.exit(0)
else:
    print(f"\nFavoured percentage ({favoured_percent}%) is < 90%.")
    print("Proceeding with SWISS-MODEL homology modeling...\n")

# --- 3. SCRIPT SETUP ---
BASE_URL = "https://swissmodel.expasy.org"
HEADERS = {"Authorization": f"Token {API_TOKEN}"}

if "YOUR_API_TOKEN_HERE" in API_TOKEN:
    print("Error: Please replace 'YOUR_API_TOKEN_HERE' with your actual API token.")
    sys.exit(1)

# --- 4. Read and PARSE the FASTA file for single or multiple sequences ---
try:
    sequences = []
    current_sequence = []
    
    with open(FASTA_FILE_PATH, 'r') as f:
        for line in f:
            line = line.strip()
            
            # If we encounter a header line
            if line.startswith('>'):
                # If we have a sequence accumulated, save it
                if current_sequence:
                    sequences.append("".join(current_sequence))
                    current_sequence = []
            else:
                # Add sequence line (ignore empty lines)
                if line:
                    current_sequence.append(line)
        
        # Don't forget the last sequence
        if current_sequence:
            sequences.append("".join(current_sequence))
    
    if not sequences:
        print(f"Error: No valid sequences found in '{FASTA_FILE_PATH}'.")
        print("Please ensure it is a valid FASTA file.")
        sys.exit(1)
    
    # Validate sequences
    for i, seq in enumerate(sequences):
        if len(seq) < 50:
            print(f"Warning: Sequence {i+1} is very short ({len(seq)} residues).")
    
    print(f"Successfully read and parsed FASTA file: {FASTA_FILE_PATH}")
    print(f"Number of sequences found: {len(sequences)}")
    
    # If single sequence, pass as string; if multiple, pass as list
    if len(sequences) == 1:
        FASTA_INPUT = sequences[0]
        print(f"Single sequence detected with {len(FASTA_INPUT)} residues")
    else:
        FASTA_INPUT = sequences
        print(f"Multiple sequences detected: {[len(s) for s in sequences]} residues each")

except FileNotFoundError:
    print(f"Error: File not found at '{FASTA_FILE_PATH}'.")
    print("Please check the path and filename.")
    sys.exit(1)
except Exception as e:
    print(f"Error reading file: {e}")
    sys.exit(1)


# --- 5. SUBMIT MODELING JOB ---
print(f"\nSubmitting job '{PROJECT_TITLE}' to SWISS-MODEL...")
payload = {
    "target_sequences": FASTA_INPUT,  # Now handles both string and list
    "project_title": PROJECT_TITLE
}

# Use the /automodel/ endpoint
try:
    submit_response = requests.post(
        f"{BASE_URL}/automodel/", 
        headers=HEADERS, 
        json=payload
    )
    submit_response.raise_for_status() # Raises an error for 4xx or 5xx responses

    project_id = submit_response.json().get("project_id")
    print(f"Job submitted successfully! Project ID: {project_id}")

except requests.exceptions.HTTPError as e:
    print(f"Error submitting job: {e.response.status_code}")
    print(f"Details: {e.response.text}")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)


# --- 6. POLL FOR RESULTS ---
while True:
    print("Checking job status...")
    try:
        # Use the /project/{project_id}/models/summary/ endpoint
        status_response = requests.get(
            f"{BASE_URL}/project/{project_id}/models/summary/", 
            headers=HEADERS
        )
        status_response.raise_for_status()
        
        status_data = status_response.json()
        job_status = status_data.get("status")
        
        print(f"Current status: {job_status}")

        if job_status == "COMPLETED":
            print("Modeling completed!")
            
            # --- 7. DOWNLOAD THE .PDB FILE ---
            models = status_data.get("models")
            if not models:
                print("Job completed but no models were found.")
                break
            
            # Get the first (and usually best) model's ID
            model_id = models[0].get("model_id")
            output_filename = f"{project_id}_{model_id}.pdb"
            
            print(f"Downloading model {model_id} to {output_filename}...")
            
            # Use the /project/{project_id}/models/{model_id}.pdb endpoint
            pdb_response = requests.get(
                f"{BASE_URL}/project/{project_id}/models/{model_id}.pdb",
                headers=HEADERS
            )
            pdb_response.raise_for_status()
            
            # Save the file
            with open(output_filename, "w") as f:
                f.write(pdb_response.text)
                
            print(f"\nSuccessfully saved model to: {output_filename}")
            print(f"You can now use this '{output_filename}' file for Step 5 (upload to CASTp).")
            
            break # Exit the while loop
            
        elif job_status == "FAILED":
            print("Job failed. Check the SWISS-MODEL website for details.")
            break # Exit the while loop
            
        elif job_status in ["RUNNING", "PENDING"]:
            print("Job is still running. Waiting 60 seconds...")
            time.sleep(60)
        
        else:
            print(f"Unknown status: {job_status}. Waiting...")
            time.sleep(60)

    except requests.exceptions.HTTPError as e:
        print(f"Error checking status: {e.response.status_code}. Retrying...")
        time.sleep(30)
    except Exception as e:
        print(f"An unexpected error occurred: {e}. Retrying...")
        time.sleep(30)

Checking Favoured percentage from CSV...
Favoured percentage found: 95.264%

Favoured percentage (95.264%) is >= 90%.
SWISS-MODEL execution is NOT required. Exiting.


SystemExit: 0

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# ========== CONFIG ==========
PDB_PATH = "proteins/5FDQ_chainA_clean.pdb"
OUTPUT_DIR = "prankweb_results"
PRANKWEB_URL = "https://prankweb.cz/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# ============================

# Get absolute path
absolute_path = os.path.abspath(PDB_PATH)
print(f"[INFO] Using absolute path: {absolute_path}")

# Setup Chrome driver with download preferences and HEADLESS mode
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new')  # Use new headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920,1080')  # Set viewport size
prefs = {
    "download.default_directory": os.path.abspath(OUTPUT_DIR),
    "download.prompt_for_download": False,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)

try:
    print("[INFO] Opening PrankWeb...")
    driver.get(PRANKWEB_URL)
    time.sleep(3)
    
    # Click "Custom structure" radio button using JavaScript
    print("[INFO] Clicking 'Custom structure' radio button...")
    wait = WebDriverWait(driver, 30)
    custom_structure = wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Custom structure')]")))
    driver.execute_script("arguments[0].click();", custom_structure)
    time.sleep(1)
    
    # Upload file with absolute path
    print(f"[INFO] Uploading: {absolute_path}")
    file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file']")
    file_input.send_keys(absolute_path)
    time.sleep(2)
    
    # Click submit using JavaScript
    print("[INFO] Submitting...")
    submit_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[type='submit']")))
    driver.execute_script("arguments[0].click();", submit_btn)
    
    # Wait for results (adjust timeout as needed)
    print("[INFO] Waiting for results (this may take several minutes)...")
    wait_long = WebDriverWait(driver, 600)  # 10 minutes
    info_tab = wait_long.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Info')]")))
    
    print("[INFO] Results ready! Clicking Info tab...")
    driver.execute_script("arguments[0].click();", info_tab)
    time.sleep(2)
    
    # Click download button using JavaScript
    print("[INFO] Clicking download button...")
    download_btn = wait_long.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Download prediction data')]")))
    driver.execute_script("arguments[0].click();", download_btn)
    
    print(f"[SUCCESS] Download started! Check {OUTPUT_DIR} folder")
    time.sleep(10)  # Wait for download to complete
    
except Exception as e:
    print(f"[ERROR] {e}")
    import traceback
    traceback.print_exc()
    
finally:
    driver.quit()
    print("Done!")

[INFO] Using absolute path: /Users/akshathr/Clones/Drugs/proteins/5FDQ_chainA_clean.pdb
[INFO] Opening PrankWeb...
[INFO] Clicking 'Custom structure' radio button...
[INFO] Uploading: /Users/akshathr/Clones/Drugs/proteins/5FDQ_chainA_clean.pdb
[INFO] Submitting...
[INFO] Waiting for results (this may take several minutes)...
[INFO] Results ready! Clicking Info tab...
[INFO] Clicking download button...
[SUCCESS] Download started! Check prankweb_results folder
Done!


In [ ]:
"""
======================================================================
ENHANCED PROTEIN PREPARATION WITH MEEKO (ADFR-Compatible Charges)
======================================================================
This script prepares protein structures for AutoDock Vina docking with:
1. PDB-Tools for robust cleaning and chain selection
2. PDBFixer for structure repair and hydrogen addition
3. Explicit histidine protonation state control (essential pre-step)
4. Meeko for reliable PDBQT conversion, charge assignment, and atom typing
5. Automated PDBQT validation and inspection

Requirements:
    pip install pdbfixer openmm rdkit pdb-tools meeko
    # Open Babel is NO LONGER REQUIRED for this version.
    
Note: Meeko's charge assignment is based on the robust
      AutoDock force field, replacing the problematic Open Babel Gasteiger step.
"""

import os
import sys
import subprocess
from pathlib import Path
import shutil

# --- Core Dependencies ---
try:
    from pdbfixer import PDBFixer
    from openmm.app import PDBFile
except ImportError:
    print("ERROR: PDBFixer or OpenMM not installed.")
    print("Install with: pip install pdbfixer openmm")
    sys.exit(1)

# --- New Meeko Dependency ---
try:
    from meeko import PDBQTWriter, MoleculePreparation
    from meeko.pdbqt_writer import _get_ad_atom_type
except ImportError:
    print("ERROR: Meeko not installed.")
    print("Install with: pip install meeko")
    sys.exit(1)

# RDKit is still useful for charge clearing but not mandatory for the Meeko path
try:
    from rdkit import Chem
except ImportError:
    print("WARNING: RDKit not installed. The charge clearing step will be skipped.")


class ProteinPreparator:
    """Enhanced protein preparation using Meeko for PDBQT conversion."""
    
    def __init__(self, input_pdb, output_dir="prepared_protein", keep_chains=None,
                 keep_residues=None, ph=7.0, his_protonation='HID', 
                 remove_heterogens=True, add_missing_atoms=True):
        """
        Initialize the enhanced protein preparator
        """
        self.input_pdb = Path(input_pdb)
        if not self.input_pdb.exists():
            raise FileNotFoundError(f"Input PDB file not found: {input_pdb}")
            
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        self.keep_chains = keep_chains
        self.keep_residues = set(keep_residues) if keep_residues else set()
        self.ph = ph
        self.his_protonation = his_protonation
        self.remove_heterogens = remove_heterogens
        self.add_missing_atoms = add_missing_atoms
        self.charge_method = 'MEEKO_ADFR_FF' # Fixed charge method for Meeko
        
        # Define output file paths
        self.pdb_tools_cleaned = self.output_dir / "step1_pdb_tools_cleaned.pdb"
        self.chain_selected = self.output_dir / "step2_chain_selected.pdb"
        self.hetatm_removed = self.output_dir / "step3_hetatm_removed.pdb"
        self.fixed_pdb = self.output_dir / "step4_structure_fixed.pdb"
        self.his_renamed = self.output_dir / "step5_his_renamed.pdb"
        self.with_hydrogens = self.output_dir / "step6_with_hydrogens.pdb"
        # self.sanitized_pdb is no longer needed
        self.final_pdbqt = self.output_dir / "protein_prepared.pdbqt"
        
        self.fixer = None
    
    def check_dependencies(self):
        """Check if all required CLI tools are available"""
        print("\n=== Checking Dependencies ===")
        
        pdb_tools = ['pdb_selchain', 'pdb_delhetatm', 'pdb_tidy']
        missing_tools = []
        for tool in pdb_tools:
            if not shutil.which(tool):
                missing_tools.append(tool)
        
        if missing_tools:
            print(f"⚠ Missing pdb-tools: {', '.join(missing_tools)}")
            print("Install with: pip install pdb-tools")
            return False
        else:
            print("✓ pdb-tools available")
        
        print("✓ Meeko (internal charge/type assignment) available")
        
        return True
    
    # ... (Steps 1, 2, 3, 4, 5, 6 remain the same using the PDBFixer output) ...
    # We copy the original step methods here for completeness, but they are identical
    
    def step1_pdb_tools_cleanup(self):
        """Step 1: Initial cleanup with pdb-tools"""
        print("\n=== Step 1: PDB-Tools Initial Cleanup ===")
        try:
            cmd = f"pdb_tidy {self.input_pdb}"
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            if result.returncode == 0:
                with open(self.pdb_tools_cleaned, 'w') as f:
                    f.write(result.stdout)
                print(f"✓ PDB cleaned with pdb_tidy")
                print(f"  Output: {self.pdb_tools_cleaned}")
                return True
            else:
                print(f"⚠ pdb_tidy failed, using original file: {result.stderr}")
                shutil.copy(self.input_pdb, self.pdb_tools_cleaned)
                return True
        except Exception as e:
            print(f"⚠ PDB-Tools cleanup failed, using original file: {e}")
            shutil.copy(self.input_pdb, self.pdb_tools_cleaned)
            return True

    def step2_select_chains(self):
        """Step 2: Select specific chains if requested"""
        print("\n=== Step 2: Chain Selection ===")
        if self.keep_chains:
            try:
                chain_arg = ','.join(self.keep_chains)
                cmd = f"pdb_selchain -{chain_arg} {self.pdb_tools_cleaned}"
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
                if result.returncode == 0:
                    with open(self.chain_selected, 'w') as f:
                        f.write(result.stdout)
                    print(f"✓ Selected chains: {', '.join(self.keep_chains)}")
                    print(f"  Output: {self.chain_selected}")
                    return True
                else:
                    print(f"⚠ Chain selection failed, keeping all: {result.stderr}")
                    shutil.copy(self.pdb_tools_cleaned, self.chain_selected)
                    return True
            except Exception as e:
                print(f"⚠ Chain selection error, keeping all: {e}")
                shutil.copy(self.pdb_tools_cleaned, self.chain_selected)
                return True
        else:
            print("  No chain selection requested, keeping all chains")
            shutil.copy(self.pdb_tools_cleaned, self.chain_selected)
            return True

    def step3_remove_heterogens(self):
        """Step 3: Remove HETATM records with pdb_delhetatm"""
        print("\n=== Step 3: Removing Heterogens ===")
        if self.remove_heterogens:
            try:
                cmd = f"pdb_delhetatm {self.chain_selected}"
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
                if result.returncode == 0:
                    with open(self.hetatm_removed, 'w') as f:
                        f.write(result.stdout)
                    print(f"✓ Removed HETATM records")
                    print(f"  Output: {self.hetatm_removed}")
                    return True
                else:
                    print(f"⚠ HETATM removal failed, keeping originals: {result.stderr}")
                    shutil.copy(self.chain_selected, self.hetatm_removed)
                    return True
            except Exception as e:
                print(f"⚠ HETATM removal error, keeping originals: {e}")
                shutil.copy(self.chain_selected, self.hetatm_removed)
                return True
        else:
            print("  Skipping heterogen removal")
            shutil.copy(self.chain_selected, self.hetatm_removed)
            return True

    def step4_pdbfixer_repair(self):
        """Step 4: Use PDBFixer to find and add missing residues/atoms"""
        print("\n=== Step 4: PDBFixer Structure Repair ===")
        self.fixer = PDBFixer(filename=str(self.hetatm_removed))
        residues = list(self.fixer.topology.residues())
        atoms = list(self.fixer.topology.atoms())
        print(f"✓ Loaded structure\n  Residues: {len(residues)}\n  Atoms: {len(atoms)}")
        
        self.fixer.findMissingResidues()
        missing_residues = self.fixer.missingResidues
        print(f"✓ Found {len(missing_residues)} missing residue segments")
        
        # Simple fix: Add missing atoms
        if self.add_missing_atoms:
            self.fixer.findMissingAtoms()
            if self.fixer.missingAtoms:
                self.fixer.addMissingAtoms()
                print("✓ Added missing atoms")
        
        # Save fixed structure (without hydrogens yet)
        PDBFile.writeFile(
            self.fixer.topology, self.fixer.positions, open(str(self.fixed_pdb), 'w'), keepIds=True
        )
        print(f"✓ Fixed structure saved: {self.fixed_pdb}")
        return True

    def step5_rename_histidines(self):
        """Step 5: Explicitly rename histidines to HID/HIE/HIP"""
        print(f"\n=== Step 5: Setting Histidine Protonation ({self.his_protonation}) ===")
        self.fixer = PDBFixer(filename=str(self.fixed_pdb))
        his_count = 0
        for residue in self.fixer.topology.residues():
            if residue.name == 'HIS':
                residue.name = self.his_protonation
                his_count += 1
        
        if his_count > 0:
            print(f"✓ Renamed {his_count} HIS residues to {self.his_protonation}")
            print(f"  HID: δ-protonated (Nδ1-H)\n  HIE: ε-protonated (Nε2-H)\n  HIP: doubly protonated (both)")
        else:
            print("  No histidine residues found")
        
        PDBFile.writeFile(
            self.fixer.topology, self.fixer.positions, open(str(self.his_renamed), 'w'), keepIds=True
        )
        print(f"✓ Saved: {self.his_renamed}")
        return True

    def step6_add_hydrogens(self):
        """Step 6: Add hydrogen atoms at specified pH"""
        print(f"\n=== Step 6: Adding Hydrogens (pH {self.ph}) ===")
        self.fixer.addMissingHydrogens(self.ph)
        
        atoms = list(self.fixer.topology.atoms())
        heavy_atoms = sum(1 for atom in atoms if atom.element.symbol != 'H')
        hydrogen_atoms = sum(1 for atom in atoms if atom.element.symbol == 'H')
        
        print(f"✓ Added {hydrogen_atoms} hydrogen atoms")
        print(f"  Total atoms: {len(atoms)} ({heavy_atoms} heavy + {hydrogen_atoms} H)")
        
        PDBFile.writeFile(
            self.fixer.topology, self.fixer.positions, open(str(self.with_hydrogens), 'w'), keepIds=True
        )
        print(f"✓ Saved: {self.with_hydrogens}")
        return True
    
    # --- Step 7: RDKit sanitization is removed as Meeko handles the PDB directly ---
    
    def step7_meeko_conversion(self):
        """
        Step 7: Convert to PDBQT using Meeko with ADFR-compatible charges.
        Meeko handles atom typing, charge assignment, and PDBQT formatting.
        """
        print(f"\n=== Step 7: Meeko Conversion with {self.charge_method} Charges ===")
        
        try:
            # 1. Load PDB into RDKit to clear formal charges (optional, but good practice)
            # This is safer than relying on RDKit's full sanitization and is kept
            # as a standalone step for charge cleaning before Meeko.
            
            # --- RDKit Charge Clearing Sub-Step ---
            mol = Chem.MolFromPDBFile(str(self.with_hydrogens), removeHs=False, sanitize=False)
            if mol is not None:
                for atom in mol.GetAtoms():
                    if atom.GetFormalCharge() != 0:
                        atom.SetFormalCharge(0)
                
                # Write to a temporary file for Meeko to read
                temp_pdb_for_meeko = self.output_dir / "temp_meeko_input.pdb"
                writer = Chem.PDBWriter(str(temp_pdb_for_meeko))
                writer.write(mol)
                writer.close()
                input_file = temp_pdb_for_meeko
                print("✓ RDKit cleared formal charges for Meeko input.")
            else:
                input_file = self.with_hydrogens
                print("⚠ RDKit failed to load PDB for charge clearing. Using PDBFixer output directly.")
            # --- End RDKit Sub-Step ---

            # 2. Prepare molecule with Meeko (Handles atom typing and charges)
            prep = MoleculePreparation(
                rigid_receptor=True,
                add_nonstd_hydrogens=False, # Hydrogens were added by PDBFixer
                preserve_graph=False,
                flexible_residues=[],
                hydrogens_model='meeko', # Ensures meeko handles H atom typing correctly
            )
            
            # Meeko's `prepare_pdb_file` handles PDB reading, atom typing, and charge assignment
            mol_set = prep.prepare_pdb_file(str(input_file))
            
            # 3. Write PDBQT file
            pdbqt_writer = PDBQTWriter(
                rigid_receptor=True,
                output_path=str(self.final_pdbqt)
            )
            pdbqt_writer.write(mol_set)
            
            # Clean up temp file
            if 'temp_pdb_for_meeko' in locals() and temp_pdb_for_meeko.exists():
                 os.remove(temp_pdb_for_meeko)

            if self.final_pdbqt.exists():
                size = self.final_pdbqt.stat().st_size
                print(f"✓ PDBQT conversion successful with Meeko")
                print(f"  Output: {self.final_pdbqt}")
                print(f"  File size: {size} bytes")
                
                return True
            else:
                print("✗ Meeko PDBQT file not created")
                return False
                
        except Exception as e:
            print(f"✗ Meeko conversion error: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def step8_inspect_pdbqt(self):
        """
        Step 8: Inspect the PDBQT output for quality control (Updated for Meeko)
        """
        print("\n=== Step 8: PDBQT Quality Inspection ===")
        
        if not self.final_pdbqt.exists():
            print("✗ PDBQT file not found")
            return False
        
        try:
            with open(self.final_pdbqt, 'r') as f:
                lines = f.readlines()
            
            # Count different record types
            atom_lines = [l for l in lines if l.startswith('ATOM') or l.startswith('HETATM')]
            
            print(f"✓ PDBQT file loaded: {len(lines)} total lines")
            print(f"  ATOM/HETATM records: {len(atom_lines)}")
            
            # Check atom types and charges
            atom_types = set()
            charges = []
            
            for line in atom_lines:
                # Atom type is at columns 77-78 (0-indexed: 77:79)
                if len(line) > 77:
                    atom_type = line[77:79].strip()
                    if atom_type:
                        atom_types.add(atom_type)
                
                # Partial charge is at columns 70-76 (0-indexed: 70:76)
                if len(line) > 70:
                    try:
                        charge_str = line[70:76].strip()
                        if charge_str:
                            charge = float(charge_str)
                            charges.append(charge)
                    except ValueError:
                        pass
            
            print(f"\n✓ Found {len(atom_types)} unique AutoDock atom types:")
            print(f"  {', '.join(sorted(atom_types))}")
            
            # Analyze charges
            if charges:
                print(f"\n✓ Partial charges detected ({self.charge_method}):")
                print(f"  Total atoms with charges: {len(charges)}")
                print(f"  Charge range: [{min(charges):.4f}, {max(charges):.4f}]")
                
                sum_charges = sum(charges)
                print(f"  Sum of charges: {sum_charges:.4f}")
                print(f"  Mean charge: {sum_charges/len(charges):.4f}")
                
                # Show charge distribution
                positive = sum(1 for c in charges if c > 0.01)
                negative = sum(1 for c in charges if c < -0.01)
                neutral = len(charges) - positive - negative
                print(f"  Distribution:")
                print(f"    Positive (>+0.01): {positive} atoms")
                print(f"    Negative (<-0.01): {negative} atoms")
                print(f"    Near-neutral: {neutral} atoms (should be low)")
                
                # Check for zero charges (which was the Open Babel issue)
                zero_charges = sum(1 for c in charges if abs(c) < 0.0001)
                if zero_charges > len(charges) * 0.9:
                    print("⚠ WARNING: Most charges are still near zero (possible Meeko failure).")
                else:
                    print("✓ Meeko generated a wide distribution of non-zero charges.")

            else:
                print(f"\n⚠ No partial charges found in PDBQT file")
            
            # Check for common issues
            issues = []
            if not charges or zero_charges > len(charges) * 0.9:
                issues.append("Partial charges are missing or mostly zero (CRITICAL ISSUE).")
            if not any(l.startswith('TORSDOF') for l in lines):
                issues.append("No TORSDOF record (expected for rigid receptor)")
            
            if issues:
                print(f"\n⚠ Potential issues found:")
                for issue in issues:
                    print(f"  • {issue}")
            else:
                print(f"\n✓ No obvious issues detected")
            
            # Show sample ATOM records with charges
            print(f"\n--- First 5 ATOM records (with charges) ---")
            for i, line in enumerate(atom_lines[:5]):
                print(f"  {line.rstrip()}")
                if len(line) > 70:
                    charge_str = line[70:76].strip()
                    atom_type = line[77:79].strip() if len(line) > 77 else ''
                    print(f"    → Charge: {charge_str}, Type: {atom_type}")
            
            print(f"\n✓ PDBQT inspection complete")
            return True
            
        except Exception as e:
            print(f"✗ Inspection failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    
    def prepare_protein(self):
        """Run the complete enhanced protein preparation pipeline with Meeko charges"""
        print(f"\n{'='*70}")
        print("ENHANCED PROTEIN PREPARATION WITH MEEKO (ADFR-Compatible Charges)")
        print(f"{'='*70}")
        print(f"Input PDB: {self.input_pdb}")
        print(f"Output directory: {self.output_dir}")
        print(f"pH: {self.ph}")
        print(f"Histidine protonation: {self.his_protonation}")
        print(f"Charge method: {self.charge_method}")
        if self.keep_chains:
            print(f"Keeping chains: {', '.join(self.keep_chains)}")

        if not self.check_dependencies():
            print("\n✗ Missing dependencies, cannot proceed")
            return None
        
        try:
            # Pipeline steps
            self.step1_pdb_tools_cleanup()
            self.step2_select_chains()
            self.step3_remove_heterogens()
            self.step4_pdbfixer_repair()
            self.step5_rename_histidines()
            self.step6_add_hydrogens()
            
            # Open Babel step is replaced by Meeko conversion
            if not self.step7_meeko_conversion():
                print("\n✗ PDBQT conversion failed")
                return None
            
            # Inspection step is moved and updated for Meeko
            self.step8_inspect_pdbqt()
            
            print(f"\n{'='*70}")
            print("✓ PROTEIN PREPARATION COMPLETE!")
            print(f"{'='*70}")
            print(f"\nFinal PDBQT file ready for docking:")
            print(f"  📄 {self.final_pdbqt}")
            print(f"  ⚡ Charges: {self.charge_method}")
            print("\nNext step:")
            print(f"  • Use {self.final_pdbqt.name} with AutoDock Vina.")
            
            return self.final_pdbqt
            
        except Exception as e:
            print(f"\n✗ ERROR during preparation: {e}")
            import traceback
            traceback.print_exc()
            return None

# Example usage
if __name__ == "__main__":
    print("\n" + "🧬 " * 25)
    print("\nEXAMPLE: Protein preparation with Meeko (ADFR-Compatible Charges)")
    print("-" * 70)
    
    # You will need to ensure 'proteins/5FDQ_chainA_clean.pdb' exists
    # Or replace it with your own PDB file path
    
    prep = ProteinPreparator(
        input_pdb="proteins/5FDQ_chainA_clean.pdb",
        output_dir="prepared_protein_meeko",
        keep_chains=['A'],
        ph=7.0,
        his_protonation='HID',
        remove_heterogens=True,
        add_missing_atoms=True,
    )
    
    # Run preparation
    result = prep.prepare_protein()
    
    if result:
        # Display protein information (requires PDBFixer object, run after prepare)
        prep.get_protein_info()
    
    print("\n" + "="*70 + "\n")
    print("MEEKO: The preferred tool for reliable AutoDock Vina preparation.")
    print("="*70)


🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 🧬 

EXAMPLE 1: Basic protein preparation with Gasteiger charges
----------------------------------------------------------------------

ENHANCED PROTEIN PREPARATION WITH GASTEIGER CHARGES
Input PDB: proteins/5FDQ_chainA_clean.pdb
Output directory: prepared_protein_gasteiger
pH: 7.0
Histidine protonation: HID
Charge method: GASTEIGER
Keeping chains: A

=== Checking Dependencies ===
✓ pdb-tools available
✓ Open Babel available: Open Babel 3.1.0 -- Oct 17 2023 -- 21:40:19

=== Step 1: PDB-Tools Initial Cleanup ===
✓ PDB cleaned with pdb_tidy
  Output: prepared_protein_gasteiger/step1_pdb_tools_cleaned.pdb

=== Step 2: Chain Selection ===
✓ Selected chains: A
  Output: prepared_protein_gasteiger/step2_chain_selected.pdb

=== Step 3: Removing Heterogens ===
✓ Removed HETATM records
  Output: prepared_protein_gasteiger/step3_hetatm_removed.pdb

=== Step 4: PDBFixer Structure Repair ===
✓ Loaded structure
  Residues: 551
  Atoms: 4473
✓ Found 2

In [6]:
import subprocess

# Direct command line equivalent
cmd = [
    'mk_prepare_receptor.py',
    '-i', 'proteins/5FDQ_chainA_clean.pdb',
    '-o', 'prepared_protein_meeko/prepared_protein',  # Don't include .pdbqt, script adds it
    '-p',
    '--charge_model', 'gasteiger',
    '--default_altloc', 'A'
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print("Error:", result.stderr)


Files written:
prepared_protein_meeko/prepared_protein.pdbqt <-- static (i.e., rigid) receptor input file



In [18]:
import pandas as pd
from vina import Vina
import os

# Input files
csv_file = "prankweb_results/prankweb-5FDQ_chainA_clean/structure.pdb_predictions.csv"
protein_pdbqt = "prepared_protein_meeko/prepared_protein.pdbqt"
ligand_pdbqt = "pdbqt/SNVK-125.pdbqt"

# Load pocket table
df = pd.read_csv(csv_file)

# Ensure output folder exists
output_dir = "docking_results"
os.makedirs(output_dir, exist_ok=True)

# Iterate over each pocket
for index, row in df.iterrows():
    pocket_name = row['name     ']
    center = [float(row['   center_x']), float(row['   center_y']), float(row['   center_z'])]

    print(f"\n===== Docking into {pocket_name} at center {center} =====")

    # Initialize Vina
    v = Vina()
    
    v.set_receptor(rigid_pdbqt_filename=protein_pdbqt)
    v.set_ligand_from_file(ligand_pdbqt)

    # Define docking box (you can adjust size if needed)
    v.compute_vina_maps(center=center, box_size=[25, 25, 25])

    # Perform docking
    v.dock(exhaustiveness=8, n_poses=10)

    # Get scores
    scores = v.energies(n_poses=10)

    # Print results
    print("Docking Results:")
    print("=" * 50)
    for i, score in enumerate(scores, 1):
        print(f"Pose {i}: Binding Affinity = {score[0]:.3f} kcal/mol")
    print("=" * 50)
    print(f"Best Binding Score: {scores[0][0]:.3f} kcal/mol")

    # Save poses for this pocket
    pose_file = os.path.join(output_dir, f"{pocket_name}_docked_poses.pdbqt")
    v.write_poses(pose_file, n_poses=10, overwrite=True)

    print(f"Poses saved to {pose_file}\n")



===== Docking into pocket1   at center [20.2974, 15.2405, 67.6991] =====
Computing Vina grid ... Docking Results:
Pose 1: Binding Affinity = -6.851 kcal/mol
Pose 2: Binding Affinity = -6.347 kcal/mol
Pose 3: Binding Affinity = -6.039 kcal/mol
Pose 4: Binding Affinity = -5.893 kcal/mol
Pose 5: Binding Affinity = -5.862 kcal/mol
Pose 6: Binding Affinity = -4.001 kcal/mol
Best Binding Score: -6.851 kcal/mol
done.
Performing docking (random seed: -132166388) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -6.851          0          0
   2       -6.347       2.39       6.42
   3       -6.039      2.407      6.506
   4       -5.893       4.52      7.241
   5       -5.862      9.851      11.39
   6       -4.001       2.72       6.62
   7       -3